# Combine DLC positions with my tracklets to impute missing values

In [46]:
%load_ext autoreload
%autoreload 2

from DLC_for_WBFM.utils.projects.utils_project import load_config, safe_cd
from pathlib import Path
import zarr
import os
import pandas as pd
import numpy as np
import napari
from DLC_for_WBFM.utils.visualization.napari_from_config import dlc_to_napari_tracks
from scipy.spatial import distance
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

from DLC_for_WBFM.utils.postprocessing.combine_tracklets_and_DLC_tracks import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# project_fname = r"Y:\shared_projects\wbfm\dlc_stacks\Charlie-worm3-new-seg\project_config.yaml"
project_fname = r"C:\dlc_stacks\Charlie-worm3-new-seg\project_config.yaml"
project_cfg = load_config(project_fname)
segment_fname = project_cfg['subfolder_configs']['segmentation']
project_dir = Path(project_fname).parent

tracklet_fname = os.path.join('2-training_data', 'all_tracklets.h5')
dlc_fname = os.path.join('3-tracking', 'full_3d_tracks.h5')

with safe_cd(project_dir):
    segment_cfg = load_config(segment_fname)
    df_tracklet = pd.read_hdf(tracklet_fname)
    dlc_tracks = pd.read_hdf(dlc_fname)

In [3]:
dlc_tracks.replace(0, np.NaN, inplace=True)

In [4]:
red_data_fname = project_cfg['preprocessed_red']
red_data = zarr.open(red_data_fname)

In [5]:
napari_tracklets = dlc_to_napari_tracks(df_tracklet, 0.0)

In [6]:
# Visualize everything
# viewer = napari.view_image(red_data)
# viewer.add_tracks(napari_tracklets)

In [17]:
all_dlc_names = list(dlc_tracks.columns.levels[0])
dlc_name = all_dlc_names[0]

all_covering_dist = []
all_covering_time_points = []
all_covering_ind = []
for i, dlc_name in enumerate(tqdm(all_dlc_names)):
    dist = calc_dlc_to_tracklet_distances(dlc_tracks, df_tracklet, dlc_name, all_covering_ind)
    covering_time_points, covering_ind, these_dist = calc_covering_from_distances(dist, df_tracklet, all_covering_ind, d_max=5)
    
    all_covering_dist.append(these_dist)
    all_covering_time_points.append(covering_time_points)
    all_covering_ind.append(covering_ind)
    if i > 3:
        break

  0%|          | 0/74 [00:00<?, ?it/s]

  0%|          | 0/5550 [00:00<?, ?it/s]

Covering of length 785 made from 8 tracklets


  0%|          | 0/5550 [00:00<?, ?it/s]

Covering of length 309 made from 5 tracklets


...

Covering of length 914 made from 8 tracklets


  0%|          | 0/5550 [00:00<?, ?it/s]

Covering of length 654 made from 18 tracklets


  0%|          | 0/5550 [00:00<?, ?it/s]

Covering of length 1012 made from 53 tracklets


In [8]:
# Distances
# %matplotlib widget
# plt.plot(all_covering_dist[0])
# np.min(all_covering_dist[0])

In [18]:
# Visualize coverings DLC + tracklets
i = 2
t_tracklet_covering = all_covering_time_points[i]
# ind_covering = all_covering_ind[i]
df_dlc = dlc_tracks[all_dlc_names[i]]
is_nan = df_dlc['x'].isnull()
t = df_dlc.index
t_dlc_covering = list(t[~is_nan])

# Process into: both, one, or neither
t_both = []
t_tracklet_only = []
t_dlc_only = []
t_gap = []
set_dlc, set_tracklet = set(t_dlc_covering), set(t_tracklet_covering)

for i in t:
    in_dlc = i in set_dlc
    in_tracklet = i in set_tracklet
    if in_dlc and in_tracklet:
        t_both.append(i)
    elif in_dlc:
        t_dlc_only.append(i)
    elif in_tracklet:
        t_tracklet_only.append(i)
    else:
        t_gap.append(i)
        

In [19]:
# %matplotlib widget
# plt.plot(t_both, 1.3*np.ones_like(t_both), 'o', label=f'Covered by both: {len(t_both)}')
# plt.plot(t_dlc_only, 1.2*np.ones_like(t_dlc_only), 'o', label=f'Points covered by only DLC: {len(t_dlc_only)}')
# plt.plot(t_tracklet_only, 1.1*np.ones_like(t_tracklet_only), 'o', label=f'Points covered by only tracklets: {len(t_tracklet_only)}')
# plt.plot(t_gap, np.ones_like(t_gap), 'o', label=f'Gaps: {len(t_gap)}')
# plt.legend()

In [20]:
%matplotlib widget
plt.scatter(t_both, 1.3*np.ones_like(t_both), label=f'Covered by both: {len(t_both)}')
plt.scatter(t_dlc_only, 1.2*np.ones_like(t_dlc_only), label=f'Points covered by only DLC: {len(t_dlc_only)}')
plt.scatter(t_tracklet_only, 1.1*np.ones_like(t_tracklet_only), label=f'Points covered by only tracklets: {len(t_tracklet_only)}')
plt.scatter(t_gap, np.ones_like(t_gap), label=f'Gaps: {len(t_gap)}')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [81]:
combined_df, new_tracklet_df = combine_all_dlc_and_tracklet_coverings(all_covering_ind, df_tracklet, dlc_tracks, verbose=0)

# Then cast as napari format
napari_new_tracklets = dlc_to_napari_tracks(combined_df, 0.0)

# ALso get original DLC tracks
dlc_name = all_dlc_names[0]
napari_dlc_tracks = dlc_to_napari_tracks(dlc_tracks[[dlc_name]], 0.0)

(1498, 3)
(1498, 3)
(1498, 3)
(1498, 3)
(1498, 3)
(1500, 3)


In [63]:
# Visualize both tracklets on the data

viewer = napari.view_image(red_data)
viewer.add_tracks(napari_new_tracklets)
viewer.add_tracks(napari_dlc_tracks)

<Tracks layer 'napari_dlc_tracks' at 0x218103f94c8>

In [56]:
tracklet_names = df_tracklet.columns.levels[0]
name = tracklet_names[all_covering_ind[0]]

'neuron14693'

In [79]:
combined_df

neuron1550                                    neuron1673              \
              z           x           y likelihood          z           x   
0      8.446753  498.350649  275.957143   1.000000  16.000000         NaN   
1      8.444737  484.360526  289.947368   1.000000  16.000000         NaN   
2      6.431944  480.381944  283.933333   1.000000  16.000000         NaN   
3      8.410738  472.357047  267.971812   1.000000  16.000000         NaN   
4      8.450135  486.397574  247.900270   1.000000  16.000000         NaN   
...         ...         ...         ...        ...        ...         ...   
1493   9.000000  442.724243  121.256971   0.464794  14.331606  400.227979   
1494   9.000000  380.768382  141.335902   0.835202  14.377632  346.305263   
1495   9.000000  310.213247  186.365255   0.718304  14.408560  288.334630   
1496   9.000000  293.140793  208.934059   0.536002  14.415716  276.310520   
1497   9.000000  291.674042  229.097054   0.762107  14.400485  276.184466   

                            neuron2224                                     \
               y likelihood          z           x           y likelihood   
0            NaN        NaN  12.417526  408.305412  289.944588   1.000000   
1            NaN        NaN  12.387054  394.272127  263.984148   1.000000   
2            NaN        NaN  12.773228  396.313386  261.979528   1.000000   
3            NaN        NaN  12.391304  388.272379  243.929668   1.000000   
4            NaN        NaN  12.419562  386.323037  247.944659   1.000000   
...          ...        ...        ...         ...         ...        ...   
1493  127.979275        1.0  15.000000         NaN         NaN        NaN   
1494  171.969737        1.0  15.000000         NaN         NaN        NaN   
1495  213.957198        1.0  15.000000  396.490784  229.568054   0.404601   
1496  239.956907        1.0  15.000000  388.118805  243.148697   0.825055   
1497  259.979369        1.0  15.000000  401.882416  246.101685   0.621020   

     neuron3529                                    neuron3796              \
              z           x           y likelihood          z           x   
0          23.0         NaN         NaN        NaN  24.000000         NaN   
1          23.0         NaN         NaN        NaN  24.000000         NaN   
2          23.0         NaN         NaN        NaN  24.000000         NaN   
3          23.0  405.668030  220.334396   0.447244  24.000000  325.147430   
4          23.0         NaN         NaN        NaN  24.000000         NaN   
...         ...         ...         ...        ...        ...         ...   
1493       23.0         NaN         NaN        NaN  20.422306  500.293233   
1494       23.0  394.227158  221.123260   0.564065  20.447471  500.363165   
1495       23.0  349.613922  250.367935   0.452385  20.372497  458.246996   
1496       23.0         NaN         NaN        NaN  22.428771  448.240223   
1497       23.0  354.748962  282.645233   0.592020  22.527299  446.228448   

                             
               y likelihood  
0            NaN        NaN  
1            NaN        NaN  
2            NaN        NaN  
3     280.525686   0.652781  
4            NaN        NaN  
...          ...        ...  
1493  265.863409   1.000000  
1494  241.948119   1.000000  
1495  225.954606   1.000000  
1496  213.909218   1.000000  
1497  215.811782   1.000000  

[1498 rows x 20 columns]

In [80]:
dlc_tracks

bodyparts neuron1550                            neuron1673              \
coords    likelihood           y           x  z likelihood           y   
0           0.740738  276.730062  498.915873  9        NaN         NaN   
1           0.723168  290.520874  484.520248  9        NaN         NaN   
2           0.480928  284.265335  482.337982  9        NaN         NaN   
3           0.757765  269.164436  473.891083  9        NaN         NaN   
4                NaN         NaN         NaN  9        NaN         NaN   
...              ...         ...         ... ..        ...         ...   
1495        0.718304  186.365255  310.213247  9   0.605029  212.819534   
1496        0.536002  208.934059  293.140793  9        NaN         NaN   
1497        0.762107  229.097054  291.674042  9   0.735358  259.859314   
1498             NaN         NaN         NaN  9   0.749943  269.122490   
1499        0.435736  236.807693  286.846710  9   0.408717  264.846802   

bodyparts                 neuron2224              ... neuron5464      \
coords              x   z          z           x  ... likelihood   z   
0                 NaN  16         15  408.135300  ...   0.542205  16   
1                 NaN  16         15         NaN  ...        NaN  16   
2                 NaN  16         15  401.117706  ...   0.451664  16   
3                 NaN  16         15  388.741608  ...        NaN  16   
4                 NaN  16         15  387.717026  ...        NaN  16   
...               ...  ..        ...         ...  ...        ...  ..   
1495       289.320282  16         15  396.490784  ...   0.634361  16   
1496              NaN  16         15  388.118805  ...        NaN  16   
1497       275.382039  16         15  401.882416  ...   0.601713  16   
1498       276.371101  16         15  401.865570  ...   0.627490  16   
1499       270.329163  16         15  387.199478  ...   0.666500  16   

bodyparts neuron5465                                       neuron67  \
coords             z           y           x likelihood           x   
0                 21         NaN         NaN        NaN  413.120300   
1                 21         NaN         NaN        NaN         NaN   
2                 21  107.917740  767.385193   0.570298  396.202118   
3                 21         NaN         NaN        NaN         NaN   
4                 21  212.024414  524.624390   0.415886         NaN   
...              ...         ...         ...        ...         ...   
1495              21  220.797836  275.477509   0.717515         NaN   
1496              21         NaN         NaN        NaN         NaN   
1497              21  268.646606  267.849747   0.674644  396.664490   
1498              21         NaN         NaN        NaN         NaN   
1499              21  273.584900  267.409302   0.617089  387.699972   

bodyparts                             
coords              y likelihood   z  
0          317.544464   0.403798  18  
1                 NaN        NaN  18  
2          294.027313   0.454399  18  
3                 NaN        NaN  18  
4                 NaN        NaN  18  
...               ...        ...  ..  
1495              NaN        NaN  18  
1496              NaN        NaN  18  
1497       220.583725   0.619689  18  
1498              NaN        NaN  18  
1499       227.496496   0.600636  18  

[1500 rows x 296 columns]

In [82]:
new_tracklet_df

neuron1550                                    neuron1673              \
              z           x           y likelihood          z           x   
0      8.446753  498.350649  275.957143        1.0        NaN         NaN   
1      8.444737  484.360526  289.947368        1.0        NaN         NaN   
2      6.431944  480.381944  283.933333        1.0        NaN         NaN   
3      8.410738  472.357047  267.971812        1.0        NaN         NaN   
4      8.450135  486.397574  247.900270        1.0        NaN         NaN   
...         ...         ...         ...        ...        ...         ...   
1493        NaN         NaN         NaN        NaN  14.331606  400.227979   
1494        NaN         NaN         NaN        NaN  14.377632  346.305263   
1495        NaN         NaN         NaN        NaN  14.408560  288.334630   
1496        NaN         NaN         NaN        NaN  14.415716  276.310520   
1497        NaN         NaN         NaN        NaN  14.400485  276.184466   

                            neuron2224                                     \
               y likelihood          z           x           y likelihood   
0            NaN        NaN  12.417526  408.305412  289.944588        1.0   
1            NaN        NaN  12.387054  394.272127  263.984148        1.0   
2            NaN        NaN  12.773228  396.313386  261.979528        1.0   
3            NaN        NaN  12.391304  388.272379  243.929668        1.0   
4            NaN        NaN  12.419562  386.323037  247.944659        1.0   
...          ...        ...        ...         ...         ...        ...   
1493  127.979275        1.0        NaN         NaN         NaN        NaN   
1494  171.969737        1.0        NaN         NaN         NaN        NaN   
1495  213.957198        1.0        NaN         NaN         NaN        NaN   
1496  239.956907        1.0        NaN         NaN         NaN        NaN   
1497  259.979369        1.0        NaN         NaN         NaN        NaN   

     neuron3529                    neuron3796                          \
              z   x   y likelihood          z           x           y   
0           NaN NaN NaN        NaN        NaN         NaN         NaN   
1           NaN NaN NaN        NaN        NaN         NaN         NaN   
2           NaN NaN NaN        NaN        NaN         NaN         NaN   
3           NaN NaN NaN        NaN        NaN         NaN         NaN   
4           NaN NaN NaN        NaN        NaN         NaN         NaN   
...         ...  ..  ..        ...        ...         ...         ...   
1493        NaN NaN NaN        NaN  20.422306  500.293233  265.863409   
1494        NaN NaN NaN        NaN  20.447471  500.363165  241.948119   
1495        NaN NaN NaN        NaN  20.372497  458.246996  225.954606   
1496        NaN NaN NaN        NaN  22.428771  448.240223  213.909218   
1497        NaN NaN NaN        NaN  22.527299  446.228448  215.811782   

                 
     likelihood  
0           NaN  
1           NaN  
2           NaN  
3           NaN  
4           NaN  
...         ...  
1493        1.0  
1494        1.0  
1495        1.0  
1496        1.0  
1497        1.0  

[1498 rows x 20 columns]

In [104]:
# %matplotlib widget
# # # from ipywidgets import interact
# best_tracklets = np.argpartition(all_means, 10)[:10]
# print(best_tracklets)
# plt.hist([all_means[i] for i in best_tracklets])

In [64]:
d = all_dist[0]
np.round(d[~np.isnan(d)], 1)

array([1.1, 0.8, 3.2, 2. , 1.2, 2.4, 0.6, 2.8, 1.7, 1.2, 0.8, 0.9, 2.9,
       0.6, 2.2, 3.7, 0.6, 0.6, 1.6, 2.2])

In [67]:
name = 'neuron1'
this_tracklet = df_tracklet[name][coords]
this_tracklet.iloc[1:20, :]

coords,z,x,y
1,8.444737,484.360526,289.947368
2,6.431944,480.381944,283.933333
3,8.410738,472.357047,267.971812
4,8.450135,486.397574,247.900270
5,8.447958,490.371542,235.891963
6,10.436702,480.283391,217.919861
7,8.451697,452.390339,199.921671
8,8.459060,446.424161,187.893960
9,8.443312,432.313376,177.970701
10,8.426020,406.274235,177.982143


In [68]:
this_dlc.iloc[1:20, :]

coords,z,x,y
1,9,484.520248,290.520874
2,9,482.337982,284.265335
3,9,473.891083,269.164436
4,9,NaN,NaN
5,9,491.375468,235.572243
6,9,482.158783,218.350449
7,9,452.336416,199.883942
8,9,449.116119,187.569519
9,9,430.832611,178.534673
10,9,405.493876,178.703758


In [73]:
len(best_tracklets)

5550